# 9.1 特殊方法

开头和结尾都是两个下划线的方法，如\_\_future\_\_，\_\_init\_\_

# 9.2 构造函数

属于特殊方法，命名为\_\_init\_\_，构造函数不同于普通方法的地方在于，将在对象创建后自动调用它们。

In [1]:
class FooBar:
    def __init__(self):
        self.somevar = 42
f = FooBar()
print(f.somevar)

42


In [5]:
class FooBar:
    def __init__(self, value=42):
        self.somevar = value
f = FooBar('hello')
print(f.somevar)
# value是可选的
f = FooBar()
print(f.somevar)

hello
42


In [8]:
class FooBar:
    def __init__(self, value):
        self.somevar = value
f = FooBar('hello')
print(f.somevar)

hello


【注】Python提供了魔法方法__del__，也称作析构函数(destructor)。这个方法在对象被销毁 (作为垃圾被收集)前被调用，但鉴于你无法知道准确的调用时间，建议尽可能不要使用\_\_del\_\_。

### 9.2.1 重写普通方法和特殊的构造函数

继承中，每个类都有一个或多个超类，并从它们那里继承行为。对类B的实例调用方法(或访问其属性)时，如果找不到该方法(或属性)，将在其超类A中查找。

In [9]:
class A:
    def hello(self):
        print("Hello, I'm A.")
class B(A): 
    pass

a = A()
b = B()
a.hello()
b.hello()

Hello, I'm A.
Hello, I'm A.


重写超类中的普通方法

In [11]:
class A:
    def hello(self):
        print("Hello, I'm A.")
class B(A):
    def hello(self):
        print("Hello, I'm B.")

a = A()
b = B()
a.hello()
b.hello()

Hello, I'm A.
Hello, I'm B.


重写超类中的构造方法

对大多数子类来说，除超类的初始化代码外，还需要有自己的初始化代码。虽然所有 方法的重写机制都相同，但与重写普通方法相比，重写构造函数时更有可能遇到一个特别的问题: 重写构造函数时，必须调用超类(继承的类)的构造函数，否则可能无法正确地初始化对象。有两种方法:1.调用未关联的超类构造函数（旧版本），2.使用函数super（新版本）。

### 9.2.2 调用未关联的超类构造函数

In [14]:
class Bird:
    def __init__(self):
        self.hungry = True 
    def eat(self):
        if self.hungry: 
            print('Aaaah ...') 
            self.hungry = False
        else:
            print('No, thanks!')
    
class SongBird(Bird): 
    def __init__(self):
        Bird.__init__(self)  # 这里
        self.sound = 'Squawk!' 
    def sing(self):
        print(self.sound)

### 9.2.3 使用函数 super

In [16]:
class Bird:
    def __init__(self):
        self.hungry = True 
    def eat(self):
        if self.hungry: 
            print('Aaaah ...') 
            self.hungry = False
        else:
            print('No, thanks!')
    
class SongBird(Bird): 
    def __init__(self):
        super.__init__()  # 这里
        self.sound = 'Squawk!' 
    def sing(self):
        print(self.sound)

函数super很聪明，因此即便有多个超类，也只需调用函数super一次(条件是所有超类的构造函数也使用函数super)。

# 9.3 元素访问

让你的自定义类可以像序列或映射那样使用

### 9.3.1 基本的序列和映射协议

序列和映射基本上是元素(item)的集合，要实现它们的基本行为(协议)，不可变对象需要实现下面的前2个方法，而可变对象需要实现下面的4个。

- __len__(self):这个方法应返回集合包含的项数，对序列来说为元素个数，对映射来说为键-值对数。如果__len__返回零(且没有实现覆盖这种行为的__nonzero__)，对象在布尔上下文中将被视为假(就像空的列表、元组、字符串和字典一样)。

- __getitem__(self, key):这个方法应返回与指定键相关联的值。对序列来说，键应该是 0~n-1 的整数(也可以是负数，这将在后面说明)，其中n为序列的长度。对映射来说，键可以是任何类型。

- __setitem__(self, key, value):这个方法应以与键相关联的方式存储值，以便以后能够使用__getitem__来获取。当然，仅当对象可变时才需要实现这个方法。

- __delitem__(self, key):这个方法在对对象的组成部分使用__del__语句时被调用，应删除与key相关联的值。同样，仅当对象可变(且允许其项被删除)时，才需要实现这个方法。

对于这些方法，还有一些额外的要求。

- 对于序列，如果键为负整数，应从末尾往前数。换而言之，x[-n]应与x[len(x)-n]等效。 

- 如果键的类型不合适(如对序列使用字符串键)，可能引发TypeError异常。

- 对于序列，如果索引的类型是正确的，但不在允许的范围内，应引发IndexError异常。

### 9.3.2 从list、dict、str派生

基本的序列/映射协议指定的4个方法能够让你走很远，但序列还有很多其他有用的魔法方法和普通方法，其中包括方法__iter__。要实现所有这些方法，不仅工作量大，而且难度不小。如果只想定制某种操作的行为，就没有理由去重新实现其他所有方法。而是使用【继承】。

# 9.4 特性

### 9.4.1 函数 property

针对类中有关联的属性，使用property减少重复的set、get代码

In [27]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def set_size(self, size):
        self.width, self.height = size 
    def get_size(self):
        return self.width, self.height
    size = property(get_size, set_size)
    
r = Rectangle()
r.set_size((1,2))
print(r.get_size())
print(r.width)
print(r.height)
print(r.size)

(1, 2)
1
2
(1, 2)


### 9.4.2 静态方法和类方法

静态方法的定义中没有参数self，可直接通过类来调用。类方法的定义中包含类似于self的参数，通常被命名为cls。对于类方法，也可通过对象直接调用，但参数cls将自动关联到类。

Python 2.4 之前

In [29]:
class MyClass:
    def smeth():
        print("This is a static method")
    smeth = staticmethod(smeth)
    
    def cmeth(cls):
        print("This is a class method of",cls)
    cmeth = classmethod(cmeth)

像这样手工包装和替换方法有点繁琐。在Python 2.4中，引入了一种名为【装饰器】的新语法

In [32]:
class MyClass:
    @staticmethod 
    def smeth():
        print('This is a static method')
    @classmethod
    def cmeth(cls):
        print('This is a class method of', cls)

定义这些方法后，就可像下面这样使用它们(无需实例化类):

In [33]:
MyClass.smeth()
MyClass.cmeth()

This is a static method
This is a class method of <class '__main__.MyClass'>


### 9.4.3 __getattr__、__setattr__等方法

- __getattribute__(self, name):在属性被访问时自动调用(只适用于新式类)。 

- __getattr__(self, name):在属性被访问而对象没有这样的属性时自动调用。

- __setattr__(self, name, value):试图给属性赋值时自动调用。 

- __delattr__(self, name):试图删除属性时自动调用。

# 9.5 迭代器

魔法方法\_\_iter\_\_是迭代器协议的基础。

### 9.5.1 迭代器协议

迭代(iterate)意味着重复多次，就像循环那样。前面只使用for循环迭代过序列和字典，但实际上也可迭代其他对象:实现了方法\_\_iter\_\_的对象。方法\_\_iter\_\_返回一个迭代器，它是包含方法\_\_next\_\_的对象，而调用这个方法时可不提供任何参数。

这有什么意义呢?为何不使用列表呢?因为在很多情况下，使用列表都有点像用大炮打蚊 子。例如，如果你有一个可逐个计算值的函数，你可能只想逐个地获取值，而不是使用列表一次 性获取。这是因为如果有很多值，列表可能占用太多的内存。但还有其他原因:使用迭代器更通用、更简单、更优雅。下面来看一个不能使用列表的示例，因为如果使用，这个列表的长度必须是无穷大的!

In [2]:
# 斐波那契数列的迭代器
class Fibs:
    def __init__(self):
        self.a = 0
        self.b = 1
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a 
    def __iter__(self):
        print(self)
        return self


【注】更正规的定义是，实现了方法\_\_iter\_\_的对象是可迭代的，而实现了方法\_\_next\_\_的对象是迭代器。

In [3]:
fibs = Fibs()
for f in fibs:
    if f > 10:
        print(f)
        break

13


【提示】通过对可迭代对象调用内置函数iter，可获得一个迭代器。

In [5]:
it = iter([1,2,3])
print(type(it))
print(it)
print(next(it))
print(next(it))

<class 'list_iterator'>
1
2


### 9.5.2 从迭代器创建序列

使用构造函数list显式地将迭代器转换为列表

In [12]:
it = iter([1,2,3])
print(list(it))

print("---")

it = iter([1,2,3])
print(next(it))
print(list(it))

print("---")

it = iter([1,2,3])
print(next(it))
print(next(it))
print(list(it))

[1, 2, 3]
---
1
[2, 3]
---
1
2
[3]


# 9.6 生成器

包含yield语句的函数都被称为生成器。这可不仅仅是 名称上的差别，生成器的行为与普通函数截然不同。差别在于，生成器不是使用return返回一个值，而是可以生成多个值，每次一个。每次使用yield生成一个值后，函数都将冻结，即在此停止执行，等待被重新唤醒。被重新唤醒后，函数将从停止的地方开始继续执行。

虽然生成器让你能够编写出非常优雅的代码，但，无论编写什么程序，都完全可以不使用生成器。

例：斐波那契数列的不同实现

A. 简单输出斐波那契數列前 N 个数

In [25]:
def fab(max): 
    n, a, b = 0, 0, 1 
    while n < max: 
        print(b,end=" ")
        a, b = b, a + b 
        n = n + 1
fab(5)

1 1 2 3 5 

结果没有问题，但有经验的开发者会指出，直接在 fab 函数中用 print 打印数字会导致该函数可复用性较差，因为 fab 函数返回 None，其他函数无法获得该函数生成的数列。

要提高 fab 函数的可复用性，最好不要直接打印出数列，而是返回一个 List。以下是 fab 函数改写后的第二个版本：

B.输出斐波那契数列前 N 个数

In [27]:
def fab(max): 
    n, a, b = 0, 0, 1 
    L = [] 
    while n < max: 
        L.append(b) 
        a, b = b, a + b 
        n = n + 1 
    return L
 
for n in fab(5): 
    print(n,end=" ")

1 1 2 3 5 

改写后的 fab 函数通过返回 List 能满足复用性的要求，但是更有经验的开发者会指出，该函数在运行中占用的内存会随着参数 max 的增大而增大，如果要控制内存占用，最好不要用 List 来保存中间结果，而是通过 iterable 对象来迭代。

C.通过 iterable 对象来迭代

In [33]:
class Fab(object): 
 
    def __init__(self, max): 
        self.max = max 
        self.n, self.a, self.b = 0, 0, 1 
 
    def __iter__(self): 
        return self 
 
    def __next__(self): 
        if self.n < self.max: 
            r = self.b 
            self.a, self.b = self.b, self.a + self.b 
            self.n = self.n + 1 
            return r 
        raise StopIteration()
 
for n in Fab(5): 
    print(n,end=" ")

1 1 2 3 5 

这个版本代码远远没有第一版的 fab 函数来得简洁。如果我们想要保持第一版 fab 函数的简洁性，同时又要获得 iterable 的效果，yield 就派上用场了：

D.使用 yield 的第四版

In [36]:
def fab(max): 
    n, a, b = 0, 0, 1 
    while n < max: 
        yield b      # 使用 yield
        # print b 
        a, b = b, a + b 
        n = n + 1
 
for n in fab(5): 
    print(n,end=" ")

1 1 2 3 5 